In [37]:
import numpy as np
import matplotlib.pyplot as plt

In [38]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def load_data_from_folder(folder_path, target_size=(128, 128)):
    images = []
    labels = []
    
    # Iterate over subdirectories for each class
    for subdir in os.listdir(folder_path):
        subdir_path = os.path.join(folder_path, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, filename)
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image = load_img(img_path, target_size=target_size)
                    image = img_to_array(image) / 255.0  # Normalize to [0, 1]
                    images.append(image)
                    labels.append(subdir)  # Use subdirectory name as label
    
    return np.array(images), np.array(labels)

# Define paths
train_data_path = r'C:\Users\ranwi\OneDrive\Desktop\Train_Data'


# Load data
train_images, train_labels = load_data_from_folder(train_data_path)
 

print(f"Training images shape: {train_images.shape}")
print(f"Training labels shape: {train_labels.shape}")



Training images shape: (2516, 128, 128, 3)
Training labels shape: (2516,)


In [39]:
def load_images_from_folder(folder_path, target_size=(128, 128)):
    images = []
    
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image = load_img(img_path, target_size=target_size)
            image = img_to_array(image) / 255.0  # Normalize to [0, 1]
            images.append(image)
    
    return np.array(images)

# Define path
test_data_path = r'C:\Users\ranwi\OneDrive\Desktop\Test_Data'

# Load test data
test_images = load_images_from_folder(test_data_path)

print(f"Test images shape: {test_images.shape}")


Test images shape: (478, 128, 128, 3)


In [40]:
def unet_model(input_shape=(128, 128, 3), num_classes=5):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(c7)  # Adjusted to output num_classes

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [41]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Convert labels to one-hot encoding
num_classes = len(label_encoder.classes_)
train_labels_one_hot = to_categorical(train_labels_encoded, num_classes=num_classes)

print(f"Encoded training labels shape: {train_labels_one_hot.shape}")


Encoded training labels shape: (2516, 5)


In [42]:
def one_hot_encode_labels(labels, num_classes):
    one_hot_labels = np.zeros((*labels.shape[:3], num_classes))
    for i in range(num_classes):
        one_hot_labels[labels == i, i] = 1
    return one_hot_labels


In [13]:
from tensorflow.keras import layers, models

def unet_model(input_shape=(128, 128, 3), num_classes=5):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    # Output layer
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [50]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [51]:
from tensorflow.keras.utils import to_categorical

# Assuming train_labels contains class indices
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_encoded = to_categorical(train_labels_encoded)  # Convert to one-hot


In [52]:

from sklearn.preprocessing import LabelEncoder
import numpy as np

# Convert string labels to integer indices
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Define one-hot encoding function
def one_hot_encode_labels(labels, num_classes=5):
    num_samples = labels.shape[0]
    one_hot_labels = np.zeros((num_samples, num_classes), dtype=np.float32)
    for i in range(num_samples):
        one_hot_labels[i, labels[i]] = 1
    return one_hot_labels

# Encode train labels to one-hot format
train_labels_one_hot = one_hot_encode_labels(train_labels_encoded, num_classes=5)

print(f"One-hot encoded train labels shape: {train_labels_one_hot.shape}")

# Check the label encoder mapping
print("Label encoding mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


One-hot encoded train labels shape: (2516, 5)
Label encoding mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'S': 4}


In [53]:
# Example dummy data for illustration purposes
def create_dummy_segmentation_labels(num_samples, height, width, num_classes):
    # Create a dummy segmentation mask
    return np.zeros((num_samples, height, width, num_classes))

# For segmentation, we need spatial dimensions for labels
train_labels_segmentation = create_dummy_segmentation_labels(train_images.shape[0], 128, 128, 5)


In [21]:
# Compile and train model
model = unet_model(input_shape=(128, 128, 3), num_classes=5)
model.compile(optimizer=Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Training with segmentation labels
history = model.fit(train_images, 
                    train_labels_segmentation, 
                    epochs=10, 
                    batch_size=16, 
                    validation_split=0.2)  # Adjust as needed

# Save the model
model.save('unet_model.h5')


Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.6066 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 238s 2s/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 3744s 30s/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 359s 3s/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 351s 3s/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 350s 3s/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 349s 3s/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 350s 3s/step - accuracy: 1.0000 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 9/10
126

In [54]:
# Save the model in the new Keras format
model.save('unet_model.keras')


In [23]:
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_45 (Conv2D)            │ (None, 128, 128, 64)      │           1,792 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_46 (Conv2D)            │ (None, 128, 128, 64)      │          36,928 │ conv2d_45[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_9               │ (None, 64, 64, 64)        │               0 │ conv2d_46[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_47 (Conv2D)            │ (None, 64, 64, 128)       │          73,856 │ max_pooling2d_9[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_48 (Conv2D)            │ (None, 64, 64, 128)       │         147,584 │ conv2d_47[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_10              │ (None, 32, 32, 128)       │               0 │ conv2d_48[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_49 (Conv2D)            │ (None, 32, 32, 256)       │         295,168 │ max_pooling2d_10[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_50 (Conv2D)            │ (None, 32, 32, 256)       │         590,080 │ conv2d_49[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_11              │ (None, 16, 16, 256)       │               0 │ conv2d_50[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_51 (Conv2D)            │ (None, 16, 16, 512)       │       1,180,160 │ max_pooling2d_11[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_52 (Conv2D)            │ (None, 16, 16, 512)       │       2,359,808 │ conv2d_51[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose_9            │ (None, 32, 32, 256)       │         524,544 │ conv2d_52[0][0]            │
│ (Conv2DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_9 (Concatenate)   │ (None, 32, 32, 512)       │               0 │ conv2d_transpose_9[0][0],  │
│                               │                           │                 │ conv2d_50[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_53 (Conv2D)            │ (None, 32, 32, 256)       │       1,179,90

 Total params: 23,092,817 (88.09 MB)

 Trainable params: 7,697,605 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 15,395,212 (58.73 MB)

In [59]:
def create_segmentation_labels(label_indices, height, width, num_classes):
    # Initialize one-hot encoded labels for segmentation
    one_hot_labels = np.zeros((label_indices.shape[0], height, width, num_classes), dtype=np.float32)
    
    for i in range(label_indices.shape[0]):
        for h in range(height):
            for w in range(width):
                # Assign class index for each pixel
                class_idx = label_indices[i]
                one_hot_labels[i, h, w, class_idx] = 1
    return one_hot_labels

# Assuming `train_labels_encoded` contains class indices for each image
train_labels_segmentation = create_segmentation_labels(train_labels_encoded, 128, 128, num_classes=5)


In [60]:
from tensorflow.keras.optimizers import Adam

model = unet_model(input_shape=(128, 128, 3), num_classes=5)

# Use categorical_crossentropy for one-hot encoded labels
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [61]:
history = model.fit(train_images, 
                    train_labels_segmentation, 
                    epochs=10, 
                    batch_size=16, 
                    validation_split=0.2)  # 20% of the data for validation


Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.3297 - loss: 1.4121 - val_accuracy: 0.5788 - val_loss: 2.9869
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.3540 - loss: 1.3383 - val_accuracy: 1.5743e-06 - val_loss: 3.1495
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.3576 - loss: 1.3204 - val_accuracy: 0.0000e+00 - val_loss: 3.5435
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.3498 - loss: 1.3295 - val_accuracy: 0.0000e+00 - val_loss: 3.8538
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1157s 9s/step - accuracy: 0.3680 - loss: 1.3244 - val_accuracy: 0.0000e+00 - val_loss: 3.4778
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1805s 14s/step - accuracy: 0.3858 - loss: 1.3215 - val_accuracy: 0.5818 - val_loss: 3.8254
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.3420 - loss: 1.3282 - val_accuracy: 0.0000e+00 - val_loss: 3.5920
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.3544 - 

In [62]:
model.save('unet_model.h5')  # Legacy format
model.save('unet_model.keras')  # Recommended format


In [63]:
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_60 (Conv2D)            │ (None, 128, 128, 64)      │           1,792 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_61 (Conv2D)            │ (None, 128, 128, 64)      │          36,928 │ conv2d_60[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_12              │ (None, 64, 64, 64)        │               0 │ conv2d_61[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_62 (Conv2D)            │ (None, 64, 64, 128)       │          73,856 │ max_pooling2d_12[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_63 (Conv2D)            │ (None, 64, 64, 128)       │         147,584 │ conv2d_62[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_13              │ (None, 32, 32, 128)       │               0 │ conv2d_63[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_64 (Conv2D)            │ (None, 32, 32, 256)       │         295,168 │ max_pooling2d_13[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_65 (Conv2D)            │ (None, 32, 32, 256)       │         590,080 │ conv2d_64[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_14              │ (None, 16, 16, 256)       │               0 │ conv2d_65[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_66 (Conv2D)            │ (None, 16, 16, 512)       │       1,180,160 │ max_pooling2d_14[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_67 (Conv2D)            │ (None, 16, 16, 512)       │       2,359,808 │ conv2d_66[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose_12           │ (None, 32, 32, 256)       │         524,544 │ conv2d_67[0][0]            │
│ (Conv2DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_12 (Concatenate)  │ (None, 32, 32, 512)       │               0 │ conv2d_transpose_12[0][0], │
│                               │                           │                 │ conv2d_65[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_68 (Conv2D)            │ (None, 32, 32, 256)       │       1,179,90

 Total params: 23,092,817 (88.09 MB)

 Trainable params: 7,697,605 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 15,395,212 (58.73 MB)

In [64]:
from tensorflow.keras.optimizers import Adam

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [67]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,          # Reduce learning rate by a factor of 0.5
    patience=3,          # Reduce after 3 epochs of no improvement
    min_lr=1e-6          # Set a minimum learning rate
)


In [68]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Manually split the data
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_segmentation, test_size=0.2, random_state=42)

# Create separate data generators for training and validation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

# Flow training data in batches using the generator
train_generator = train_datagen.flow(X_train, y_train, batch_size=16)

# Flow validation data in batches using a separate generator
val_generator = val_datagen.flow(X_val, y_val, batch_size=16)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[lr_scheduler]
)


Epoch 1/30


C:\Users\ranwi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.3055 - loss: 1.6777 - val_accuracy: 0.3698 - val_loss: 1.4601 - learning_rate: 1.0000e-04
Epoch 2/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.3681 - loss: 1.4751 - val_accuracy: 0.3710 - val_loss: 1.4585 - learning_rate: 1.0000e-04
Epoch 3/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.3542 - loss: 1.4865 - val_accuracy: 0.3710 - val_loss: 1.4533 - learning_rate: 1.0000e-04
Epoch 4/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 5495s 44s/step - accuracy: 0.3581 - loss: 1.4728 - val_accuracy: 0.3710 - val_loss: 1.4531 - learning_rate: 1.0000e-04
Epoch 5/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.3672 - loss: 1.4598 - val_accuracy: 0.3710 - val_loss: 1.4530 - learning_rate: 1.0000e-04
Epoch 6/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - accuracy: 0.3778 - loss: 1.4651 - val_accuracy: 0.3710 - val_loss: 1.4526 - learning_rate: 1.0000e-04
Epoch 7/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.